In [ ]:
# Code for Optimizing Supply Chain Cost under Trade-war

In [88]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

#Component landed cost by i: supplier, j: factory, k: component
RMCost = np.array([ [[10, 10000, 10000, 10000],
                     [11, 10000, 10000, 10000],
                     [50, 10000, 10000, 10000]],

                    [[13, 10000, 10000, 10000],
                     [9, 10000, 10000, 10000],
                     [15, 10000, 10000, 10000]],

                    [[10000, 16, 10000, 10000],
                     [10000, 17, 10000, 10000],
                     [10000, 18, 10000, 10000]],

                    [[10000, 19, 10000, 10000],
                     [10000, 20, 10000, 10000],
                     [10000, 21, 10000, 10000]],
                   
                    [[10000, 10000, 22, 10000],
                     [10000, 10000, 23, 10000],
                     [10000, 10000, 24, 10000]],
                   
                    [[10000, 10000, 25, 10000],
                     [10000, 10000, 26, 10000],
                     [10000, 10000, 27, 10000]],    
                   
                    [[10000, 10000, 10000, 40],
                     [10000, 10000, 10000, 29],
                     [10000, 10000, 10000, 30]],

                    [[10000, 10000, 10000, 31],
                     [10000, 10000, 10000, 32],
                     [10000, 10000, 10000, 33]]  ])

#Assembly cost for 3 factories, used same array structure. 
#allocate each iphone assembly cost to k components.
AssemCost = np.array([ [[1.1, 10000, 10000, 10000],
                     [2.1, 10000, 10000, 10000],
                     [3.1, 10000, 10000, 10000]],

                    [[1.1, 10000, 10000, 10000],
                     [2.1, 10000, 10000, 10000],
                     [3.1, 10000, 10000, 10000]],

                    [[10000, 1.2, 10000, 10000],
                     [10000, 2.2, 10000, 10000],
                     [10000, 3.2, 10000, 10000]],

                    [[10000, 1.2, 10000, 10000],
                     [10000, 2.2, 10000, 10000],
                     [10000, 3.2, 10000, 10000]],
                   
                    [[10000, 10000, 1.3, 10000],
                     [10000, 10000, 2.3, 10000],
                     [10000, 10000, 3.3, 10000]],
                   
                    [[10000, 10000, 1.3, 10000],
                     [10000, 10000, 2.3, 10000],
                     [10000, 10000, 3.3, 10000]],    
                   
                    [[10000, 10000, 10000, 1.4],
                     [10000, 10000, 10000, 2.4],
                     [10000, 10000, 10000, 3.4]],

                    [[10000, 10000, 10000, 1.4],
                     [10000, 10000, 10000, 2.4],
                     [10000, 10000, 10000, 3.4]]  ])


# Logistics cost of shipping 1 iphone from 3 factory to customers
Logcost = ([2, 5, 1])

# I: Number of supplier 8, J: Number of factory 3, K: Number of component 4
I, J, K = RMCost.shape

# duty rate from factory 1 (China) to US customer
duty_rate = 0.15

print(I)
print(J)
print(K)
print(RMCost[0,1,0])

8
3
4
11


In [89]:
def model_setup():
    
    m = Model("SCCost")

    # Creat variables
    x = m.addVars(I, J, K, vtype=GRB.BINARY, name = "x")
    
    # set objective with RMcost only
    m.setObjective( quicksum( ((1+duty_rate)* RMCost[i,0,k] * x[i,0,k] 
                               + RMCost[i, 1, k] * x[i,1,k] 
                               + RMCost[i, 2, k] * x[i,2,k]) for i in range(I) for k in range(K)), GRB.MINIMIZE)
    

    
    # There should be only 1 supplier for 1 component to 1 factory
    m.addConstrs( ( quicksum(x[i,j,k] for i in range(I)) == 1 for j in range(J) for k in range(K))  )
    
    # There can be max. 1 component supplied from 1 supplier to 1 factory
    m.addConstrs( ( quicksum(x[i,j,k] for k in range(K)) <= 1 for i in range(I) for j in range(J))  )
    
    #1 component can be supplied by 1 supplier to max of 3 countries (because we have 3 assemblers)
    m.addConstrs( ( quicksum(x[i,j,k] for j in range(J)) <= 3 for i in range(I) for k in range(K))  )    
    
    return m

In [90]:
# setup the model 
m_SC = model_setup()

# solving the model
m_SC.optimize()


# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
x = m_SC.getVars()

# reformat the vector as a matrix with dimension NxM
x = np.reshape(x, (I,J,K))

print("selected arcs = supplier, factory, component: RMcost")
for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x == 1:
                print(i+1, j+1, k+1, ":", RMCost[i,j,k]) 


Optimize a model with 68 rows, 96 columns and 288 nonzeros
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 84608.300000
Presolve removed 56 rows and 64 columns
Presolve time: 0.00s
Presolved: 12 rows, 32 columns, 64 nonzeros
Found heuristic solution: objective 255.8500000
Variable types: 0 continuous, 32 integer (32 binary)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 255.85 20242.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.558500000000e+02, best bound 2.558500000000e+02, gap 0.0000%
selected arcs = supplier, factory, component: RMcost
1 1 1 : 10
2 2 1 : 9
2 3 1 : 15
3 1 2 : 16
3 2 2 : 17
3 3 2 : 18
5 1 3 : 22
5 2 3 : 23
5 3 3 : 24
7 2 4 : 29
7 3 4 : 30
8 1 4 : 31


In [91]:
# Phase 2 - adding Factory Assembly cost

# change objective to include Factory Assembly cost
m_SC.setObjective( quicksum( ((1+duty_rate)* (RMCost[i,0,k] + AssemCost[i,0,k])* x[i,0,k] 
                               + (RMCost[i, 1, k]+ AssemCost[i,1,k])* x[i,1,k] 
                               + (RMCost[i, 2, k]+ AssemCost[i,2,k]) * x[i,2,k]) for i in range(I) for k in range(K)), GRB.MINIMIZE)
# solving the model
m_SC.optimize()

# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
x = m_SC.getVars()

# reformat the vector as a matrix with dimension NxM
x = np.reshape(x, (I,J,K))

print("selected arcs = supplier, factory, component: RMcost")
for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x == 1:
                print(i+1, j+1, k+1, ":", RMCost[i,j,k]) 


Optimize a model with 68 rows, 96 columns and 288 nonzeros
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

Loaded MIP start with objective 283.6

Presolve removed 68 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 283.6 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.836000000000e+02, best bound 2.836000000000e+02, gap 0.0000%
selected arcs = supplier, factory, component: RMcost
1 1 1 : 10
2 2 1 : 9
2 3 1 : 15
3 1 2 : 16
3 2 2 : 17
3 3 2 : 18
5 1 3 : 22
5 2 3 : 23
5 3 3 : 24
7 2 4 : 29
7 3 4 : 30
8 1 4 : 31


In [94]:
# Phase 3 - adding Logistics cost from factory to customers
# change objective to include Logsitics cost, Logcost / K is to allocate the FG log cost to each component to avoid double count in below formular.
m_SC.setObjective( quicksum( (((1+duty_rate)* (RMCost[i,0,k] + AssemCost[i,0,k]) + Logcost[0]/K )* x[i,0,k] 
                               + (RMCost[i, 1, k]+ AssemCost[i,1,k] + Logcost[1]/K)* x[i,1,k] 
                               + (RMCost[i, 2, k]+ AssemCost[i,2,k] + Logcost[2]/K) * x[i,2,k]) for i in range(I) for k in range(K)), GRB.MINIMIZE)
# solving the model
m_SC.optimize()

# extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
x = m_SC.getVars()

# reformat the vector as a matrix with dimension NxM
x = np.reshape(x, (I,J,K))

print("selected arcs = supplier, factory, component: RMcost")
for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x == 1:
                print(i+1, j+1, k+1, ":", RMCost[i,j,k]) 

Optimize a model with 68 rows, 96 columns and 288 nonzeros
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

Loaded MIP start with objective 291.6

Presolve removed 68 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 291.6 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.916000000000e+02, best bound 2.916000000000e+02, gap 0.0000%
selected arcs = supplier, factory, component: RMcost
1 1 1 : 10
2 2 1 : 9
2 3 1 : 15
3 1 2 : 16
3 2 2 : 17
3 3 2 : 18
5 1 3 : 22
5 2 3 : 23
5 3 3 : 24
7 2 4 : 29
7 3 4 : 30
8 1 4 : 31
